In [1]:
from platform import system
from targets import target_values
from residuals import mdl_resid
from tiktak import filer
 
print('Hi!')
 
import os
os.environ['MKL_CBWR']='AUTO'

from estimates import get_point

Hi!


In [2]:
x_base, tm = get_point(True,read_wisdom=False) # high education

tar = target_values(tm)

x_comp, _ = get_point(False,read_wisdom=False) # low education
print(x_base)
print(x_comp)

{'sigma_psi': 0.02351795998155682, 'sigma_psi_init': 0.06319749482803798, 'pmeet_21': 0.36231065032307264, 'pmeet_30': 1.2185173549427066, 'pmeet_40': 1.0, 'preg_21': 0.006683178914886262, 'preg_28': 0.030446109898106186, 'preg_35': 0.026257274180916695, 'u_shift_mar': 0.24749413577501758, 'util_alp': 0.5739290298121097, 'util_kap': 8.484242183675391, 'util_qbar': 5.561954628246849, 'disutil_marry_sm_mal': 4.408415907989258, 'disutil_shotgun': 0.4715857724582857, 'abortion_costs': 1.8092065614536414, 'p_abortion_access': 0.9512267376733684, 'u_lost_divorce': 0.8892578980806901, 'mu_psi_init': -0.15876965206098093, 'high education': True}
{'sigma_psi': 0.04119975516565719, 'sigma_psi_init': 0.07184509981781, 'pmeet_21': 0.7300641341551373, 'pmeet_30': 0.38552526708748397, 'pmeet_40': 1.4132304041226518, 'preg_21': 0.1029100967053943, 'preg_28': 0.11241132276639117, 'preg_35': 0.11203564468462099, 'u_shift_mar': 0.338428482678413, 'util_alp': 0.5195282434982275, 'util_kap': 7.15239876088

In [3]:
pmeets = ['pmeet_21','pmeet_30','pmeet_40']
ppregs = ['preg_21','preg_28','preg_35']
abortions = ['abortion_costs','p_abortion_access']
shifters = ['disutil_marry_sm_mal','disutil_shotgun','u_lost_divorce','util_qbar']
prefs = ['u_shift_mar','util_alp','util_kap']
match = ['sigma_psi','sigma_psi_init','mu_psi_init']
he = ['high education']
replace_fields = [('match quality',match),
                  ('preferences',prefs),
                  ('trend only',he),
                  ('pmeets',pmeets),
                  ('ppregs',ppregs),
                  ('remar',['disutil_marry_sm_mal']),
                  ('stigma',['disutil_shotgun']),            
                  ('div costs',['u_lost_divorce']),
                  ('qbar',['util_qbar']),
                  ('abortions',abortions),                  
                  ('match and prefs',match+prefs)
                 ]

xlist = [x_base]
names = ['college']
for name, rf in replace_fields:
    xnew = x_base.copy()
    for f in rf:
        xnew[f] = x_comp[f]
    xlist.append(xnew)
    names.append(name)
    
names.append('high school')
xlist.append(x_comp)

names = ['educ comparison: '+ n for n in names]
print(names)

['educ comparison: college', 'educ comparison: match quality', 'educ comparison: preferences', 'educ comparison: trend only', 'educ comparison: pmeets', 'educ comparison: ppregs', 'educ comparison: remar', 'educ comparison: stigma', 'educ comparison: div costs', 'educ comparison: qbar', 'educ comparison: abortions', 'educ comparison: match and prefs', 'educ comparison: high school']


In [4]:
for x, name in zip(xlist,names):        
    print(name)
    #print(x)
    
    out,  mom = mdl_resid(x=x,targets=tar,
                                      return_format=['distance','moments'],
                                      verbose=False,draw=False,cs_moments=False,
                                      save_to ='mdl for {}'.format(name),
                                      moments_save_name = name,
                                      moments_repeat=5)
    
    
    
    
    


educ comparison: college
using cupy!


/projects/p30190/py_shotgun/simulations.py:612: RuntimeWarning: invalid value encountered in long_scalars
  self.share_aborted[t] = 100*n_abortions / (n_abortions + n_kept)
/projects/p30190/py_shotgun/moments.py:118: RuntimeWarning: Mean of empty slice.
  moments['ever kids by years after marriage, {}'.format(t)] = ever_kid[pick & (self.yaftmar==t)].mean()
/projects/p30190/shotgunenv/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Distance 1.6960509353824487, time 74.31
educ comparison: match quality
Distance 214.1207945011066, time 90.47
educ comparison: preferences
Distance 86.93019925767078, time 63.55
educ comparison: trend only
Distance 194.0946235902452, time 81.77
educ comparison: pmeets
Distance 28.888477184934043, time 75.95
educ comparison: ppregs
Distance 74.18140449093626, time 67.57
educ comparison: remar
Distance 2.9308342226752147, time 70.57
educ comparison: stigma
Distance 1.9582995053905208, time 71.56
educ comparison: div costs
Distance 16.233234465264605, time 72.52
educ comparison: qbar
Distance 1.6803385354328082, time 71.36
educ comparison: abortions
Distance 2.458533659005753, time 72.31
educ comparison: match and prefs
Distance 255.84424864188406, time 76.19
educ comparison: high school
Distance 199.59947821837557, time 84.11


In [12]:
# this converts moments to outputs
names_select = [('Col','educ comparison: college'),
                ('HS','educ comparison: high school'), 
                ('MQ','educ comparison: match quality'),
                ('CP','educ comparison: preferences'),
                ('CPMQ','educ comparison: match and prefs'),                
                ('I','educ comparison: trend only'),
                ('M','educ comparison: pmeets'), 
                ('P','educ comparison: ppregs'),
                ('D','educ comparison: div costs'),
                ('R','educ comparison: remar'), 
                ('S','educ comparison: stigma'),
                ('A','educ comparison: abortions'),
                ('Q','educ comparison: qbar')]

tags = ''
div_kfs = 'divorced if kids first'
div_mfs = 'divorced if marriage first'
div_rats = 'divorced ratio KF/MF'
kf_rels = 'relative KF at 35'

for tag, name in names_select:
    mom = filer(name + '.pkl',0,0,repeat=False)
    div_10_kf = mom['divorced by years after marriage if kids first, 10']
    div_10_mf = mom['divorced by years after marriage if marriage first, 10']
    share_kf_35 = mom['k then m in sample at 35']
    div_cs_kf = mom['divorced if k then m and one marriage']
    div_cs_mf = mom['divorced if m then k and one marriage']
    div_kfs += ' & {:02.1f}'.format(100*div_10_kf)
    div_mfs += ' & {:02.1f}'.format(100*div_10_mf)
    div_rats += ' & {:02.1f}'.format(div_10_kf/div_10_mf)
    kf_rels += ' & {:02.1f}'.format(100*share_kf_35)
    tags += (' & ' + tag)
    
tags += r' \\'
div_kfs += r' \\'
div_mfs += r' \\'
div_rats += r' \\'
kf_rels += r' \\'

    
print(tags)
print(div_kfs)
print(div_mfs)
print(div_rats)
print(kf_rels)
    


 & Col & HS & MQ & CP & CPMQ & I & M & P & D & R & S & A & Q \\
divorced if kids first & 21.2 & 21.7 & 43.5 & 12.7 & 31.9 & 10.0 & 21.3 & 17.7 & 32.7 & 17.8 & 18.9 & 15.3 & 21.5 \\
divorced if marriage first & 5.7 & 15.8 & 25.6 & 9.8 & 23.0 & 1.6 & 5.7 & 5.7 & 9.1 & 6.5 & 5.7 & 5.6 & 5.7 \\
divorced ratio KF/MF & 3.7 & 1.4 & 1.7 & 1.3 & 1.4 & 6.1 & 3.8 & 3.1 & 3.6 & 2.7 & 3.3 & 2.7 & 3.8 \\
relative KF at 35 & 9.2 & 33.6 & 8.4 & 7.5 & 6.5 & 24.9 & 7.0 & 30.1 & 9.5 & 9.9 & 8.7 & 8.2 & 9.2 \\


In [6]:
# this converts moments to outputs

'''
names_select = [('HS','educ comparison backwards: high school'),
                ('Col','educ comparison backwards: college'), 
                ('I','educ comparison backwards: trend only'),
                ('M','educ comparison backwards: pmeets'), 
                ('P','educ comparison backwards: ppregs'),
                ('D','educ comparison backwards: div costs'),
                ('R','educ comparison backwards: remar'), 
                ('S','educ comparison backwards: stigma'),
                ('A','educ comparison backwards: abortions'),
                ('Q','educ comparison backwards: qbar')]

tags = ''
div_kfs = 'divorced if kids first'
div_mfs = 'divorced if marriage first'
div_rats = 'divorced ratio KF/MF'
kf_rels = 'relative KF at 35'

for tag, name in names_select:
    mom = filer(name + '.pkl',0,0,repeat=False)
    div_10_kf = mom['divorced by years after marriage if kids first, 5']
    div_10_mf = mom['divorced by years after marriage if marriage first, 5']
    share_kf_35 = mom['k then m in sample at 35']
    div_cs_kf = mom['divorced if k then m and one marriage']
    div_cs_mf = mom['divorced if m then k and one marriage']
    div_kfs += ' & {:02.1f}'.format(100*div_10_kf)
    div_mfs += ' & {:02.1f}'.format(100*div_10_mf)
    div_rats += ' & {:02.1f}'.format(div_10_kf/div_10_mf)
    kf_rels += ' & {:02.1f}'.format(100*share_kf_35)
    tags += (' & ' + tag)
    
tags += r' \\'
div_kfs += r' \\'
div_mfs += r' \\'
div_rats += r' \\'
kf_rels += r' \\'

    
print(tags)
print(div_kfs)
print(div_mfs)
print(div_rats)
print(kf_rels)
'''



Problems opening the file educ comparison backwards: high school.pkl


Exception: could not open the file